## Creating simualtion of Spot Price By Monte-Carlo

In [130]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import seaborn as sns
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date




import json
with open('config.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

Trading_Days        = data['Trading_days_in_year']
Expiry_Days         = data['No_of_days']
Threshold_prob      = data['Threshold_probability']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
tenor_list      = [0.083333333333333333]
count           = 0
count_list      = []
while count < simu:
    count = count + 1
    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor      = ((21-count)/(21*12))

    
    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)
    tenor_list.append(tenor)
last_price_list.pop(-1)
tenor_list.pop(-1)

df['Count'] = count_list
df['Price'] = last_price_list
df['Change_price'] = diff_list
df['tenor'] = tenor_list


{'Current_Price': 17689, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.07, 'sigma': 0.1449, 'No_of_days': 21, 'Trading_days_in_year': 252, 'Threshold_probability': 0.36}


In [131]:

#calculation of probabiltiy of hitting nearest strike price 

Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)
df['Sigma']      = Daily_volatility*df['Price']



main_df = []
PL      =  []

y       =  0

while y < 21:

    
    price = df['Price'].iloc[0]
    df1 = pd.DataFrame()

    CE_Sell_list            =   []
    CE_Buy_list             =   []
    CE_Sell_thetalist       =   []
    CE_Buy_thetalist        =   []
    CE_prob_list            =   []

    PE_Sell_list            =   []
    PE_Sell_thetalist       =   []
    PE_prob_list            =   []

    PE_Buy_list             =   []
    PE_Buy_thetalist        =   []



    for last_price,T,sig in zip(df['Price'],df['tenor'],df['Sigma']):

        k                  = int(round(price,-2))+600
        d1                 = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2                 = d1-sigma*(T**0.5)
        N_d1               = norm.cdf(d1)
        N_d2               = norm.cdf(d2)
        N_d1_              = 1-norm.cdf(d1)
        N_d2_              = 1-norm.cdf(d2)
        CE_Sell            = last_price*N_d1-k*math.exp(-r*T)*N_d2
        CE_Sell_theta      = 0.01*((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
        Prob_Call          = norm.pdf((k-last_price)/sig)

        CE_prob_list.append(Prob_Call)
        CE_Sell_list.append(CE_Sell)
        CE_Sell_thetalist.append(CE_Sell_theta)

        


        k                  = int(round(price,-2))+900
        d1                 = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2                 = d1-sigma*(T**0.5)
        N_d1               = norm.cdf(d1)
        N_d2               = norm.cdf(d2)
        N_d1_              = 1-norm.cdf(d1)
        N_d2_              = 1-norm.cdf(d2)
        CE_Buy             = -(last_price*N_d1-k*math.exp(-r*T)*N_d2)
        CE_Buy_theta       = 0.01*((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
        
        CE_Buy_list.append(CE_Buy)
        CE_Buy_thetalist.append(CE_Buy_theta)

        
    

        p                  = int(round(price,-2))-600
        d1                 = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2                 = d1-sigma*(T**0.5)
        N_d1               = norm.cdf(d1)
        N_d2               = norm.cdf(d2)
        N_d1_              = 1-norm.cdf(d1)
        N_d2_              = 1-norm.cdf(d2)
        call               = last_price*N_d1-p*math.exp(-r*T)*N_d2
        PE_Sell            = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
        PE_Sell_theta      = 0.01*(-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
        Prob_put           = norm.pdf((p-last_price)/sig)

        PE_Sell_list.append(PE_Sell)
        PE_Sell_thetalist.append(PE_Sell_theta)
        PE_prob_list.append(Prob_put)

    
    

        p                  = int(round(price,-2))-800
        d1                 = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2                 = d1-sigma*(T**0.5)
        N_d1               = norm.cdf(d1)
        N_d2               = norm.cdf(d2)
        N_d1_              = 1-norm.cdf(d1)
        N_d2_              = 1-norm.cdf(d2)
        call               = last_price*N_d1-p*math.exp(-r*T)*N_d2
        PE_Buy             = -(p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1))
        PE_Buy_theta       = 0.01*(-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
            
        PE_Buy_list.append(PE_Buy)
        PE_Buy_thetalist.append(PE_Buy_theta)


        # if Prob_put >= 0.30 or Prob_Call >= 0.30:
        #     break

        
    df1['CE_Buy']               =   CE_Buy_list
    df1['theta_CE_Buy']         =   CE_Buy_thetalist
    df1['Prob_call']            =   CE_prob_list
    df1['CE_Sell_']             =   CE_Sell_list
    df1['theta_CE_Sell']        =   CE_Sell_thetalist
    df1['Prob_put']             =   PE_prob_list
    df1['PE_Sell']              =   PE_Sell_list
    df1['theta_PE_Sell']        =   PE_Sell_thetalist
    df1['PE_Buy']               =   PE_Buy_list
    df1['theta_PE_Buy']         =   PE_Buy_thetalist  

    #df2 = pd.concat([df.iloc[0:len(df1)-1],df1.iloc[0:len(df1)-1]],axis = 1)

    df1 = pd.concat([df,df1],axis=1)

    
    #breaking the dataframe after hitting threshold probability
    resultant_df=pd.DataFrame(columns = [i for i in df1.columns]) 
    for index,value in df1.iterrows():
        if value['Prob_put'] > Threshold_prob or value['Prob_call'] > Threshold_prob :
            break   
        else: 
            resultant_df.loc[index]=value
    resultant_df


    #Calculating the total premium paid or recieved 
    filter_col = [col for col in list(resultant_df) if col.startswith(('PE_','CE_'))]
    resultant_df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
    for d in resultant_df[filter_col]:
        resultant_df['Total_Prem'] = resultant_df[filter_col].sum(axis=1)   # Total_PP = total put premium 
        
    #calculating total Profit and Loss
    resultant_df['Total_P&L'] = resultant_df['Total_Prem'][0] - resultant_df['Total_Prem']

    PL.append(resultant_df['Total_P&L'].iloc[-1])


    df = df.iloc[len(resultant_df):len(df)]
    df.reset_index(drop =True, inplace=True)
    
    

    main_df.append(resultant_df)
    
    x =pd.concat(main_df)
    
    #x.to_csv('gandu.csv')
    
    # print(x)
    y = len(x)




In [132]:
print("Total profit after 21 days : ",sum(PL)*500)
df = pd.concat(main_df)
df.reset_index(inplace=True)


#Changing the signs of greeks according to options selling and buying
df['theta_PE_Sell'] = - df['theta_PE_Sell']
df['theta_CE_Sell'] = - df['theta_CE_Sell']

#Calculating greeks for overall data

list1 = ['theta']
datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(df) if col.startswith(data)]
    df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in df[filter_col]:
        df['Port_'+ data] = df[filter_col].sum(axis=1)    

Total profit after 21 days :  31177.266360790945


In [133]:
df.to_csv('prob.csv')

In [134]:
df

,index,Count,Price,Change_price,tenor,Sigma,CE_Buy,theta_CE_Buy,Prob_call,CE_Sell_,theta_CE_Sell,Prob_put,PE_Sell,theta_PE_Sell,PE_Buy,theta_PE_Buy,Total_Prem,Total_P&L,Port_theta
0,0,1.0,17689.000000,226.600639,0.083333,739.913659,-5.602161e+01,-1.211795e+01,0.283686,112.307829,17.432967,0.290609,66.520818,8.962372,-37.999240,-6.764242,84.807798,0.000000,7.513144
1,1,2.0,17915.600639,100.182956,0.079365,749.392143,-9.213581e+01,-1.648338e+01,0.349764,173.944780,21.574517,0.220647,34.016922,6.674763,-17.838268,-4.505564,97.987623,-13.179825,7.260334
2,0,3.0,18015.783595,-64.581816,0.075397,753.582698,-5.312629e+01,-1.271790e+01,0.295394,109.253532,18.534037,0.285704,57.780794,9.234717,-31.980737,-6.758292,81.927301,0.000000,8.292561
3,1,4.0,17951.201780,-122.766811,0.071429,750.881304,-3.984311e+01,-1.112377e+01,0.274658,86.695753,17.169763,0.304717,64.338573,10.136225,-35.573211,-7.515267,75.618008,6.309294,8.666952
4,2,5.0,17828.434969,-60.095606,0.067460,745.746088,-2.388833e+01,-8.356495e+00,0.233597,56.930506,14.253503,0.338251,83.512223,11.778135,-47.017907,-9.128254,69.536488,12.390813,8.546889
5,3,6.0,17768.339363,137.750178,0.063492,743.232347,-1.667081e+01,-6.860733e+00,0.213313,42.713223,12.595304,0.352839,92.219163,12.777665,-51.843972,-10.067175,66.417607,15.509694,8.445059
6,4,7.0,17906.089542,67.114367,0.059524,748.994303,-2.326373e+01,-9.054707e+00,0.259734,58.104548,15.773578,0.317519,59.293215,11.356921,-30.932754,-8.134802,63.201275,18.726026,9.940990
7,5,8.0,17973.203908,-108.207738,0.055556,751.801632,-2.523207e+01,-1.007920e+01,0.281820,63.762079,17.422805,0.298318,44.487579,10.507788,-21.907578,-7.062482,61.110015,20.817286,10.788915
8,6,9.0,17864.996171,175.095445,0.051587,747.275407,-1.397844e+01,-7.169558e+00,0.245944,39.856985,14.101348,0.328723,56.994500,12.535351,-28.385356,-8.759775,54.487690,27.439611,10.707366
9,7,10.0,18040.091615,-26.801483,0.047619,754.599479,-2.296076e+01,-1.068617e+01,0.302941,62.313137,19.183246,0.278397,28.411448,9.429589,-12.514981,-5.675213,55.248844,26.678458,12.251450


In [135]:
len(main_df)

3

In [136]:
# ##

# #calculation of probabiltiy of hitting nearest strike price 

# Trading_Days = 252
# Expiry_Days = 21
# Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)
# df['Sigma']      = Daily_volatility*df['Price']




# df1 = pd.DataFrame()

# CE_Sell_list            =   []
# CE_Buy_list             =   []
# CE_Sell_thetalist   =   []
# CE_Buy_thetalist   =   []
# CE_prob_list       =   []

# PE_Sell_list         =   []
# PE_Sell_thetalist      =   []
# PE_prob_list      = []

# PE_Buy_list        =   []
# PE_Buy_thetalist      =   []

# for last_price,T,sig in zip(df['Price'],df['tenor'],df['Sigma']):

#     k             = int(round(price,-2))+600
#     d1            = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2            = d1-sigma*(T**0.5)
#     N_d1          = norm.cdf(d1)
#     N_d2          = norm.cdf(d2)
#     N_d1_         = 1-norm.cdf(d1)
#     N_d2_         = 1-norm.cdf(d2)
#     CE_Sell       = last_price*N_d1-k*math.exp(-r*T)*N_d2
#     CE_Sell_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
#     Prob_Call          = norm.pdf((k-last_price)/sig)

#     CE_prob_list.append(Prob_Call)
#     CE_Sell_list.append(CE_Sell)
#     CE_Sell_thetalist.append(CE_Sell_theta)

    


#     k             = int(round(price,-2))+900
#     d1            = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2            = d1-sigma*(T**0.5)
#     N_d1          = norm.cdf(d1)
#     N_d2          = norm.cdf(d2)
#     N_d1_         = 1-norm.cdf(d1)
#     N_d2_         = 1-norm.cdf(d2)
#     CE_Buy       = last_price*N_d1-k*math.exp(-r*T)*N_d2
#     CE_Buy_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
      
#     CE_Buy_list.append(CE_Buy)
#     CE_Buy_thetalist.append(CE_Buy_theta)

    
  

#     p          = int(round(price,-2))-600
#     d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2         = d1-sigma*(T**0.5)
#     N_d1       = norm.cdf(d1)
#     N_d2       = norm.cdf(d2)
#     N_d1_      = 1-norm.cdf(d1)
#     N_d2_      = 1-norm.cdf(d2)
#     call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
#     PE_Sell        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
#     PE_Sell_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
#     Prob_put = norm.pdf((p-last_price)/sig)



#     PE_Sell_list.append(PE_Sell)
#     PE_Sell_thetalist.append(PE_Sell_theta)
#     PE_prob_list.append(Prob_put)

  
  

#     p          = int(round(price,-2))-800
#     d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2         = d1-sigma*(T**0.5)
#     N_d1       = norm.cdf(d1)
#     N_d2       = norm.cdf(d2)
#     N_d1_      = 1-norm.cdf(d1)
#     N_d2_      = 1-norm.cdf(d2)
#     call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
#     PE_Buy        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
#     PE_Buy_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
        
#     PE_Buy_list.append(PE_Sell)
#     PE_Buy_thetalist.append(PE_Sell_theta)


#     # if Prob_put >= 0.30 or Prob_Call >= 0.30:
#     #     break

    
# df1['CE_Buy']               =   CE_Buy_list
# df1['theta_CE_Buy']         =   CE_Buy_thetalist
# df1['Prob_call']            =   CE_prob_list
# df1['CE_Sell_']             =   CE_Sell_list
# df1['theta_CE_Sell']        =   CE_Sell_thetalist
# df1['Prob_put']             =   PE_prob_list
# df1['PE_Sell']              =   PE_Sell_list
# df1['theta_PE_Sell']        =   PE_Sell_thetalist
# df1['PE_Buy']               =   PE_Buy_list
# df1['theta_PE_Buy']         =   PE_Buy_thetalist  

# #df2 = pd.concat([df.iloc[0:len(df1)-1],df1.iloc[0:len(df1)-1]],axis = 1)

In [137]:
# df1 = pd.concat([df,df1],axis=1)

# resultant_df=pd.DataFrame(columns=[i for i in df1.columns]) 
# for index,value in df1.iterrows():
#     if value['Prob_put']>0.30 or value['Prob_call']>0.30:
#         break   
#     else: 
#         resultant_df.loc[index]=value
# resultant_df


# df = df.iloc[len(resultant_df):len(df)]
# df

# df.reset_index(inplace = True)

# Trading_Days = 252
# Expiry_Days = 21
# Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)
# df['Sigma']      = Daily_volatility*df['Price']




# df2 = pd.DataFrame()

# CE_Sell_list            =   []
# CE_Buy_list             =   []
# CE_Sell_thetalist   =   []
# CE_Buy_thetalist   =   []
# CE_prob_list       =   []

# PE_Sell_list         =   []
# PE_Sell_thetalist      =   []
# PE_prob_list      = []

# PE_Buy_list        =   []
# PE_Buy_thetalist      =   []

# for last_price,T,sig in zip(df['Price'],df['tenor'],df['Sigma']):

#     k             = int(round(df['Price'].iloc[0],-2))+600
#     d1            = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2            = d1-sigma*(T**0.5)
#     N_d1          = norm.cdf(d1)
#     N_d2          = norm.cdf(d2)
#     N_d1_         = 1-norm.cdf(d1)
#     N_d2_         = 1-norm.cdf(d2)
#     CE_Sell       = last_price*N_d1-k*math.exp(-r*T)*N_d2
#     CE_Sell_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
#     Prob_Call          = norm.pdf((k-last_price)/sig)

#     CE_prob_list.append(Prob_Call)
#     CE_Sell_list.append(CE_Sell)
#     CE_Sell_thetalist.append(CE_Sell_theta)

    


#     k             = int(round(df['Price'].iloc[0],-2))+900
#     d1            = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2            = d1-sigma*(T**0.5)
#     N_d1          = norm.cdf(d1)
#     N_d2          = norm.cdf(d2)
#     N_d1_         = 1-norm.cdf(d1)
#     N_d2_         = 1-norm.cdf(d2)
#     CE_Buy       = last_price*N_d1-k*math.exp(-r*T)*N_d2
#     CE_Buy_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
      
#     CE_Buy_list.append(CE_Buy)
#     CE_Buy_thetalist.append(CE_Buy_theta)

    
  

#     p          = int(round(df['Price'].iloc[0],-2))-600
#     d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2         = d1-sigma*(T**0.5)
#     N_d1       = norm.cdf(d1)
#     N_d2       = norm.cdf(d2)
#     N_d1_      = 1-norm.cdf(d1)
#     N_d2_      = 1-norm.cdf(d2)
#     call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
#     PE_Sell        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
#     PE_Sell_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
#     Prob_put = norm.pdf((p-last_price)/sig)



#     PE_Sell_list.append(PE_Sell)
#     PE_Sell_thetalist.append(PE_Sell_theta)
#     PE_prob_list.append(Prob_put)

  
  

#     p          = int(round(df['Price'].iloc[0],-2))-800
#     d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
#     d2         = d1-sigma*(T**0.5)
#     N_d1       = norm.cdf(d1)
#     N_d2       = norm.cdf(d2)
#     N_d1_      = 1-norm.cdf(d1)
#     N_d2_      = 1-norm.cdf(d2)
#     call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
#     PE_Buy        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
#     PE_Buy_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
        
#     PE_Buy_list.append(PE_Sell)
#     PE_Buy_thetalist.append(PE_Sell_theta)


#     # if Prob_put >= 0.30 or Prob_Call >= 0.30:
#     #     break

    
# df2['CE_Buy']               =   CE_Buy_list
# df2['theta_CE_Buy']         =   CE_Buy_thetalist
# df2['Prob_call']            =   CE_prob_list
# df2['CE_Sell_']             =   CE_Sell_list
# df2['theta_CE_Sell']        =   CE_Sell_thetalist
# df2['Prob_put']             =   PE_prob_list
# df2['PE_Sell']              =   PE_Sell_list
# df2['theta_PE_Sell']        =   PE_Sell_thetalist
# df2['PE_Buy']               =   PE_Buy_list
# df2['theta_PE_Buy']         =   PE_Buy_thetalist  

# #df2 = pd.concat([df.iloc[0:len(df1)-1],df1.iloc[0:len(df1)-1]],axis = 1)


# df2 = pd.concat([df,df2],axis = 1)

# resultant_df1=pd.DataFrame(columns=[i for i in df2.columns]) 
# for index,value in df2.iterrows():
#     if value['Prob_put']>0.30 or value['Prob_call']>0.30:
#         break   
#     else: 
#         resultant_df1.loc[index]=value
# resultant_df1

# df = pd.concat([resultant_df,resultant_df1])

#df.reset_index(inplace=True)

# #Changing the signs of greeks according to options selling and buying
# df['PE_Buy'] = - df['PE_Buy']
# df['CE_Buy'] = - df['CE_Buy']


# df['theta_PE_Sell'] = - df['theta_PE_Sell']

# df['theta_CE_Sell'] = - df['theta_CE_Sell']




# #Calculating the total premium paid or recieved 
# filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
# df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
# for d in df[filter_col]:
#     df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


# #Calculating the total Profit and Loss

# df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']


# #Calculating greeks for overall data

# list1 = ['theta']

# datalist4 = dict()

# for data in list1:
#     filter_col = [col for col in list(df) if col.startswith(data)]
#     df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
#     for d in df[filter_col]:
#         df['Port_'+ data] = df[filter_col].sum(axis=1)    





# df['PE_Buy'] = - df['PE_Buy']
# df['CE_Buy'] = - df['CE_Buy']







# #Calculating the total premium paid or recieved 
# filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
# df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
# for d in df[filter_col]:
#     df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

# df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']
